In [40]:
import pandas as pd
import sqlite3
from cleantext import clean

from pathlib import Path
import numpy as np
import swifter
from somajo import Tokenizer, SentenceSplitter
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()

In [41]:
conn = sqlite3.connect("/home/filter/ptf/data/million_post_corpus/corpus.sqlite3")

In [42]:
comments = pd.read_sql('select headline, body, createdat from posts', conn)

In [43]:
comments = comments.fillna('')

In [44]:
comments['text'] = comments.swifter.apply(lambda x: x['Headline'] + ' ' + x['Body'], axis=1)

In [47]:
tokenizer = Tokenizer(split_camel_case=True, token_classes=False, extra_info=False)
sentence_splitter = SentenceSplitter(is_tuple=False)
def get_sents(text):
    text = clean(text, lang='de', lower=False)
    tokens = tokenizer.tokenize_paragraph(text)
    sentences = sentence_splitter.split(tokens)
    return [clean(' '.join(s), no_urls=True, no_digits=True, no_punct=True, no_line_breaks=True, lang='de') for s in sentences]

In [48]:
results = Parallel(n_jobs=num_cores)(delayed(get_sents)(row) for row in tqdm(comments['text'].values))


100%|██████████| 1011773/1011773 [12:34<00:00, 1340.56it/s]

In [49]:
res_flat = list(np.hstack(np.array(results)))

In [50]:
res_flat = [r for r in res_flat if len(r) > 10]

In [53]:
res_flat[:50]

['newsletter der standard ich bin begeistert von den standard newslettern',
 'in keiner anderen ezeitung gibt es eine solche viel = falt und so interessante kommentare bzw eine so genaue berichterstattung',
 'macht weiter so ich finde es jedenfalls toll',
 'johann radakovits',
 'auch begeistert',
 'aber momentan funktioniert das abmelden oder adressen ändern überhaupt nicht',
 'in usa macht man dies auf den servern einfacher',
 'ansonsten ist der standard ja wirklich das beste was es in österreich gibt',
 'weiter im text carpe diem',
 'abmeldeprobleme es ist ganz einfach nervend',
 'vor kurzem habe ich den newsletter angemeldet in der hoffnungsvollen gewissheit weil vertrauenserweckend angekündigt ihn jederzeit wieder stornieren zu können',
 'aber leider führt kein weg dahin',
 'ich wäre dankbar für eine diesbezügliche hilfe des lesers dieser email',
 'und sie als mitarbeiter sind natuerlich objektiv',
 'erstklassig ist dieser newsletterservice und deshalb wird er von mir gerne weitere

In [52]:
Path('../hyperwords/ompc.txt').write_text('\n'.join(res_flat))

206140197